In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 결과지표 감소가 큰 유전자 추리기

In [4]:
import sqlite3
import pandas as pd
db_path = "/content/drive/MyDrive/BIO_MEDICAL_PROJECT/obesity_gene_analysis/data/OBESITYGENE_PROJECT.db"

In [5]:
# X 데이터프레임이 될 것
query = f"""
SELECT
    E.subject_id,
    P.gene_symbol,
    E.exp_C_A
FROM expression_change AS E
JOIN platform AS P ON E.ID_REF = P.ID
"""

conn = sqlite3.connect(db_path)
total_genes_symbols_exp_C_A_df = pd.read_sql_query(query, conn)

In [6]:
filtered_total_genes_symbols_exp_C_A_df = total_genes_symbols_exp_C_A_df[total_genes_symbols_exp_C_A_df['gene_symbol'].notnull()]

In [7]:
pivot_filtered_total_genes_symbols_exp_C_A_df = filtered_total_genes_symbols_exp_C_A_df.pivot_table(
    index='subject_id',
    columns='gene_symbol',
    values='exp_C_A')

In [8]:
pivot_filtered_total_genes_symbols_exp_C_A_df.shape

(49, 20254)

In [9]:
# y 데이터프레임이 될 것
conn = sqlite3.connect(db_path)
SHAP_weight_bmi_changes_df = pd.read_sql_query("""
    SELECT subject_id, weight_kg_C_A, bmi_C_A, fat_C_A, pure_fat_C_A
    FROM weight_bmi_changes
""", conn, index_col='subject_id')
conn.close()

In [11]:
SHAP_weight_bmi_changes_df.shape

(53, 4)

In [10]:
X=pivot_filtered_total_genes_symbols_exp_C_A_df
y=SHAP_weight_bmi_changes_df

In [12]:
# 공통 인덱스 (subject_id) 추출
common_index = X.index.intersection(y.index)

# 공통 인덱스 기준으로 X, y 정렬 및 필터링
X_common = X.loc[common_index]
y_common = y.loc[common_index]

In [13]:
print(X_common.shape)
print(y_common.shape)
print(X_common.index.equals(y_common.index))  # True여야 안전

(49, 20254)
(49, 4)
True


In [14]:
# X_common과 y_common을 좌우로 합치기 (subject_id 기준)
merged_df = pd.concat([X_common, y_common], axis=1)

# 결측치가 있는 행 제거
merged_clean = merged_df.dropna()

# 다시 X, y로 분리
X_clean = merged_clean[X.columns]
y_clean = merged_clean[y.columns]

In [15]:
print(X_clean.shape)
print(y_clean.shape)

(47, 20254)
(47, 4)


In [16]:
X_clean

,A1BG,A1CF,A2LD1,A2M,A2ML1,A4GALT,A4GNT,AAA1,AAAS,AACS,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
subject_id,,,,,,,,,,,,,,,,,,,,,
1,-0.203501,-0.257881,-0.076882,-0.070258,-0.053789,0.359254,-0.525279,-0.250846,0.081067,0.004297,...,0.203297,-0.789169,0.054097,-0.100937,-0.061814,0.180741,-0.074276,0.131013,-0.009875,0.059363
10,-0.289141,0.304956,-0.136773,-0.200091,-0.079607,-0.082132,-0.199102,-0.014800,0.011206,-0.147787,...,-0.271067,0.472637,0.128869,-0.576064,0.290886,-0.395441,0.254112,0.090162,-0.000692,-0.381745
11,-0.018572,0.201289,0.048916,0.176325,0.179518,0.082603,-0.238181,0.127768,-0.369594,-0.065608,...,-0.340940,-0.229782,-0.160442,0.269090,-0.080181,-0.094051,-0.304135,-0.041985,-0.288120,0.045331
12,-0.364803,-0.186572,0.010086,0.158080,-0.691446,0.133029,-0.342992,-0.272503,0.193401,-0.209218,...,0.237398,0.237283,0.058561,-0.223919,-0.022895,0.357639,0.042628,-0.013599,0.232416,0.234180
13,0.068879,-0.071343,-0.361604,0.083380,-0.186179,-0.366747,-0.017106,0.168394,0.034616,-0.329525,...,-0.272767,0.341191,0.093876,-0.208187,0.128282,-0.346685,-0.085323,0.542646,-0.346277,0.167544
14,0.281799,-0.355884,-0.035775,-0.153475,0.017159,-0.201125,0.173375,-0.176636,-0.500170,-0.202860,...,-0.330408,0.657490,0.085367,-0.208483,-0.136246,-0.141856,-0.175162,-0.142296,-0.021148,0.337131
18,-0.648461,-0.394088,0.078309,0.289820,-0.649276,-0.237888,-0.398332,-0.746614,-0.015811,0.081993,...,-1.397364,-0.437187,0.083598,-0.418047,0.086827,0.062638,-0.349114,0.255005,0.288087,-0.521277
19,0.743935,-0.111559,0.508087,0.021051,-0.666260,0.054175,-0.278458,0.076177,0.575289,0.613777,...,0.208418,1.111191,0.236500,-0.320331,-0.070009,0.178334,0.167641,-0.109791,-0.137900,-0.014339
2,-0.063825,-0.235455,0.045844,0.073061,0.365945,0.051688,0.310467,0.088913,-0.407162,-0.867728,...,-0.427302,-0.282632,0.320404,0.245004,-0.372646,0.189487,0.089606,-0.117723,-0.576978,-0.210223


In [17]:
y_clean

,weight_kg_C_A,bmi_C_A,fat_C_A,pure_fat_C_A
subject_id,,,,
1,-9.59,-3.44,-7.5,-9.36066
10,-10.48,-3.55,-4.4,-8.68676
11,-9.73,-3.00,-6.5,-8.44394
12,-5.14,-1.89,-2.6,-4.61656
13,-11.14,-3.91,-6.1,-11.36910
14,-10.08,-2.93,-5.8,-10.11374
18,-10.42,-3.15,-7.9,-11.21949
19,-3.93,-1.63,-1.2,-3.17421
2,-9.97,-3.70,-6.2,-9.05613


In [19]:
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
import shap

# 모델 훈련
base_model = xgb.XGBRegressor(n_estimators=100, max_depth=4, random_state=42)
model = MultiOutputRegressor(base_model)
model.fit(X_clean, y_clean)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=4,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))

만약 평균 SHAP 말고 Z score로 구한다면 추려지는 유전자 리스트가 달라지려나?-> 추후 재실험을 통해 알아보기


In [22]:
import numpy as np
# SHAP 계산
shap_all = [] # SHAP 결과 저장용 리스트

# 4개의 타겟 각각에 대해 SHAP 계산
for i, name in enumerate(y_clean.columns):
    sub_model = model.estimators_[i] # 각 타겟별 모델 추출

    explainer = shap.Explainer(sub_model, X_clean)
    shap_values = explainer(X_clean)

    # 평균 SHAP 계산
    mean_shap = np.abs(shap_values.values).mean(axis=0)  # shape: (n_features,)
    shap_all.append(mean_shap)

genes = sorted(filtered_total_genes_symbols_exp_C_A_df['gene_symbol'].unique()) # sorted(...): 유전자의 이름을 알파벳 순으로 정렬

# 4개 타겟별 평균 SHAP을 하나의 DataFrame으로 결합
shap_df = pd.DataFrame(np.array(shap_all).T, columns=[f"SHAP_{name}" for name in y_clean.columns])
shap_df['Gene'] = genes
shap_df = shap_df[['Gene'] + [f"SHAP_{name}" for name in y_clean.columns]]

In [24]:
shap_df = shap_df.set_index('Gene')

In [25]:
shap_df

,SHAP_weight_kg_C_A,SHAP_bmi_C_A,SHAP_fat_C_A,SHAP_pure_fat_C_A
Gene,,,,
A1BG,0.043611,0.009058,0.017235,0.043624
A1CF,0.008576,0.009630,0.029263,0.021776
A2LD1,0.014665,0.011567,0.028808,0.028428
A2M,0.004319,0.012313,0.011292,0.028155
A2ML1,0.007654,0.000000,0.003713,0.006418
...,...,...,...,...
ZYX,0.000000,0.000000,0.000000,0.000000
ZZEF1,0.000000,0.000000,0.000000,0.000000
ZZZ3,0.000000,0.000000,0.000000,0.000000


In [26]:
# 각 타겟별로 SHAP > 0인 유전자 수와 비율 계산
for col in shap_df.columns:
    nonzero_df = shap_df[shap_df[col] > 0]
    num_nonzero = nonzero_df.shape[0]
    total_genes = shap_df.shape[0]
    ratio = num_nonzero / total_genes
    print(f"[{col}] SHAP > 0인 유전자 수: {num_nonzero}")
    print(f"비율: {ratio:.4%}\n")

[SHAP_weight_kg_C_A] SHAP > 0인 유전자 수: 142
비율: 0.7011%

[SHAP_bmi_C_A] SHAP > 0인 유전자 수: 130
비율: 0.6418%

[SHAP_fat_C_A] SHAP > 0인 유전자 수: 138
비율: 0.6813%

[SHAP_pure_fat_C_A] SHAP > 0인 유전자 수: 138
비율: 0.6813%



In [27]:
obesity_genes = ['FTO', 'MC4R', 'LEP', 'LEPR', 'ADIPOQ', 'PPARG',
                 'UCP1', 'UCP2', 'BDNF', 'GNPDA2', 'TMEM18', 'NEGR1']

In [29]:
# 해당 유전자 필터링
shap_obesity_rank = shap_df[shap_df.index.isin(obesity_genes)]

# 원하는 SHAP 열 기준 정렬하여 출력
print(shap_obesity_rank.sort_values(by='Gene'))

        SHAP_weight_kg_C_A  SHAP_bmi_C_A  SHAP_fat_C_A  SHAP_pure_fat_C_A
Gene                                                                     
ADIPOQ                 0.0           0.0           0.0                0.0
BDNF                   0.0           0.0           0.0                0.0
FTO                    0.0           0.0           0.0                0.0
GNPDA2                 0.0           0.0           0.0                0.0
LEP                    0.0           0.0           0.0                0.0
LEPR                   0.0           0.0           0.0                0.0
MC4R                   0.0           0.0           0.0                0.0
NEGR1                  0.0           0.0           0.0                0.0
PPARG                  0.0           0.0           0.0                0.0
TMEM18                 0.0           0.0           0.0                0.0
UCP1                   0.0           0.0           0.0                0.0
UCP2                   0.0           0

In [30]:
shap_df[shap_df.columns].describe()

,SHAP_weight_kg_C_A,SHAP_bmi_C_A,SHAP_fat_C_A,SHAP_pure_fat_C_A
count,20254.000000,20254.000000,20254.000000,20254.000000
mean,0.000209,0.000064,0.000200,0.000219
std,0.006821,0.001967,0.012519,0.007395
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,0.560240,0.156273,1.597833,0.519105


In [33]:
# 각 결과지표에 대해 SHAP 값이 0보다 큰 유전자만 추출하고, SHAP 값 기준 내림차순 정렬 후 Gene 이름만 리스트로 저장
# 1. 체중(weight)
shap_weight_genes = shap_df[shap_df['SHAP_weight_kg_C_A'] > 0]\
    .sort_values(by='SHAP_weight_kg_C_A', ascending=False).index.tolist()

# 2. BMI
shap_bmi_genes = shap_df[shap_df['SHAP_bmi_C_A'] > 0]\
    .sort_values(by='SHAP_bmi_C_A', ascending=False).index.tolist()

# 3. 체지방률(fat_pct)
shap_fat_pct_genes = shap_df[shap_df['SHAP_fat_C_A'] > 0]\
    .sort_values(by='SHAP_fat_C_A', ascending=False).index.tolist()

# 4. 체지방량(fat_mass)
shap_fat_mass_genes = shap_df[shap_df['SHAP_pure_fat_C_A'] > 0]\
    .sort_values(by='SHAP_pure_fat_C_A', ascending=False).index.tolist()

In [38]:
print(len(shap_weight_genes))
print(len(shap_bmi_genes))
print(len(shap_fat_pct_genes))
print(len(shap_fat_mass_genes))

142
130
138
138


In [34]:
shap_weight_genes

['HELLS',
 'ZNF100',
 'BTN2A1',
 'CCDC13',
 'DR1',
 'TRMT5',
 'GRIA3',
 'BCAP31',
 'ZNF257',
 'C20orf71',
 'BRSK1',
 'DAG1',
 'APOBEC3B',
 'REPS2',
 'ABCB4',
 'ZNF821',
 'A4GALT',
 'NDUFAF3',
 'ABHD12',
 'CXorf40B',
 'KCTD10',
 'A1BG',
 'AASS',
 'C11orf68',
 'ABCC1',
 'ASXL3',
 'C9orf141',
 'AAAS',
 'FMNL2',
 'ABCG4',
 'ADIG',
 'AARS',
 'ACBD3',
 'NAA25',
 'ABCC10',
 'POLD3',
 'TRIM37',
 'PPAP2A',
 'MMEL1',
 'C17orf70',
 'PSMD11',
 'ABHD8',
 'HIST1H4C',
 'A2LD1',
 'ACER3',
 'ABCB8',
 'NDST4',
 'AAK1',
 'CES1',
 'MYEOV2',
 'ADK',
 'AANAT',
 'A1CF',
 'NKRF',
 'A2ML1',
 'C11orf85',
 'PLOD1',
 'IFT172',
 'FLJ11292',
 'ACSS2',
 'S100A16',
 'LTA4H',
 'APOE',
 'IMP3',
 'E2F5',
 'ECHS1',
 'ABCC6P2',
 'LEFTY1',
 'A2M',
 'ACCN1',
 'ALG3',
 'C15orf34',
 'ACO2',
 'IL1F5',
 'ADCK3',
 'SETD4',
 'PION',
 'C8orf31',
 'SNORD116-1',
 'CFH',
 'PRAMEF11',
 'PRKACA',
 'OGN',
 'BFAR',
 'ARHGAP26',
 'C22orf27',
 'MBD3L1',
 'MIR429',
 'ARHGAP22',
 'ANKRD13D',
 'ELOVL5',
 'LHB',
 'AAA1',
 'AADACL3',
 'FAM171B'

In [35]:
shap_bmi_genes

['BTN2A1',
 'C11orf68',
 'ARPP21',
 'ADPRHL1',
 'HELLS',
 'FBLN5',
 'C12orf32',
 'CLK2P',
 'ZNF821',
 'APOBEC3C',
 'REPS2',
 'RSL1D1',
 'KCNAB1',
 'SERPING1',
 'AATK',
 'TNFRSF13C',
 'KDSR',
 'ABCB4',
 'A4GNT',
 'HBG1',
 'LOXL1',
 'ACHE',
 'AADACL4',
 'PTX4',
 'ECHS1',
 'ARL4D',
 'A2M',
 'A2LD1',
 'PABPC4',
 'ABCA7',
 'ASL',
 'ABCA9',
 'DCP2',
 'VGLL3',
 'A1CF',
 'A4GALT',
 'IQCD',
 'A1BG',
 'FAM129A',
 'ACACB',
 'AAMP',
 'ABCG8',
 'ABTB2',
 'C21orf81',
 'AAK1',
 'ITGB3',
 'AANAT',
 'ZNF17',
 'ARPC3',
 'KRTAP20-2',
 'MYEOV2',
 'SNORD116-21',
 'AADACL2',
 'JMJD1C',
 'RGS11',
 'OR5P2',
 'TAF5L',
 'CDHR2',
 'AASDH',
 'ACAA1',
 'ACOT4',
 'BHLHE22',
 'DPP9',
 'ABCB10',
 'ATP2B4',
 'RPS15A',
 'FUT9',
 'MRPL23',
 'KLHL30',
 'APOE',
 'ATP6V0E1',
 'IMP3',
 'ABCC10',
 'IER2',
 'ABLIM3',
 'DUSP1',
 'MAGOHB',
 'SYNPO',
 'ABCA12',
 'C8G',
 'ALAD',
 'MIR429',
 'ARSH',
 'ABCA2',
 'CNFN',
 'ADAMTS6',
 'AAA1',
 'AARS2',
 'ATXN3L',
 'ACCN2',
 'ATAD3A',
 'EDA',
 'GPR141',
 'ARPC5L',
 'LOC100129884',
 'TM

In [36]:
shap_fat_pct_genes

['ORMDL3',
 'PPP2R1A',
 'C20orf46',
 'CSRNP3',
 'COL8A1',
 'GADD45G',
 'THPO',
 'TMEM115',
 'NUBP2',
 'MRVI1',
 'ADCY2',
 'ALDH9A1',
 'FAM86A',
 'CLPTM1',
 'A4GALT',
 'BTN3A1',
 'ARTN',
 'AGPAT6',
 'ABCC3',
 'A1CF',
 'A2LD1',
 'ANKRD20A5',
 'AAA1',
 'AATK',
 'ABHD8',
 'ARHGEF15',
 'MYLIP',
 'IER2',
 'ABCC6',
 'ACACB',
 'ACTA2',
 'FAM104A',
 'OLFM1',
 'AKAP1',
 'A1BG',
 'OR5H1',
 'ABCC11',
 'ERI2',
 'ATP8A1',
 'A2M',
 'NHS',
 'MYH4',
 'AADACL2',
 'LRRC15',
 'AASS',
 'STOML1',
 'CDK4',
 'LPAL2',
 'ABCC1',
 'MORF4L2',
 'B3GAT3',
 'C20orf111',
 'ABCG8',
 'MRPS9',
 'AANAT',
 'AAGAB',
 'ABCB7',
 'LZTS1',
 'DEPDC7',
 'COMTD1',
 'NPLOC4',
 'MGC44328',
 'ABCA8',
 'CAPN6',
 'ANKRD34C',
 'A2ML1',
 'SNX12',
 'REPS2',
 'APOBEC1',
 'AACS',
 'C20orf200',
 'ABCA9',
 'C15orf43',
 'WNK3',
 'ATP13A3',
 'ABCB10',
 'C5orf45',
 'MRPL32',
 'AURKC',
 'ZNF69',
 'MIR130A',
 'HELLS',
 'PKIG',
 'TSPAN1',
 'ABCA5',
 'PEX7',
 'TXLNG',
 'C10orf76',
 'ABCA3',
 'NLRC3',
 'ADAMTSL1',
 'TRAF4',
 'FMR1NB',
 'ABCC12',
 'H

In [37]:
shap_fat_mass_genes

['HNRNPA1P10',
 'BTN2A1',
 'PTPRD',
 'SNORD116-1',
 'CDCA5',
 'C8orf80',
 'ARF5',
 'ANP32B',
 'SULT1C4',
 'CBY1',
 'APBB2',
 'RGS11',
 'AIMP2',
 'ADIPOR2',
 'FAM116A',
 'MIR21',
 'ABCC4',
 'JUNB',
 'C19orf73',
 'LRP8',
 'NPW',
 'C1orf213',
 'A1BG',
 'ADAMTS9',
 'C5orf20',
 'AAK1',
 'ZNF362',
 'PHF1',
 'FAM86A',
 'A2LD1',
 'A2M',
 'AADAC',
 'ABCA13',
 'AAA1',
 'A1CF',
 'FAM71F2',
 'ATG9A',
 'ABCC12',
 'ANKLE1',
 'C16orf93',
 'ADPRHL1',
 'AADACL4',
 'BUD13',
 'ABCC3',
 'ALKBH6',
 'SNORD102',
 'FARS2',
 'GOLGA8G',
 'TMEFF2',
 'DCLK1',
 'A4GNT',
 'ATP2A2',
 'ARHGAP27',
 'KCP',
 'AAAS',
 'ACTL7A',
 'C9orf23',
 'A2ML1',
 'ABCB5',
 'BRSK2',
 'SNX12',
 'ABCC5',
 'C16orf78',
 'ANKFN1',
 'SERPINA10',
 'MAGEA10',
 'PAGE2',
 'KLHDC8A',
 'AANAT',
 'C15orf61',
 'FAM59B',
 'AAGAB',
 'GPR75',
 'ANKS1B',
 'RPA4',
 'HIST1H2BA',
 'FLJ35816',
 'CNOT4',
 'ATRNL1',
 'SPDEF',
 'MIR181C',
 'LY6G5C',
 'LYZL6',
 'PAX3',
 'CAPN3',
 'FITM2',
 'B3GALT5',
 'NBPF22P',
 'A4GALT',
 'ODZ2',
 'IL12B',
 'CCDC102A',
 'ARI

In [39]:
# 4개 SHAP 양수 유전자 리스트 교집합
common_genes = list(
    set(shap_weight_genes) &
    set(shap_bmi_genes) &
    set(shap_fat_pct_genes) &
    set(shap_fat_mass_genes)
)

print(f"모든 지표에서 SHAP > 0인 유전자 수: {len(common_genes)}")
print(common_genes)

모든 지표에서 SHAP > 0인 유전자 수: 7
['A1BG', 'AANAT', 'A2M', 'A4GALT', 'A1CF', 'AAA1', 'A2LD1']


In [44]:
# 4개 SHAP 양수 유전자 리스트 합집합
all_genes = list(
    set(shap_weight_genes) |
    set(shap_bmi_genes) |
    set(shap_fat_pct_genes) |
    set(shap_fat_mass_genes)
)

print(f"4개 결과지표 중 하나라도 SHAP > 0인 유전자 수: {len(all_genes)}")
print(all_genes)

4개 결과지표 중 하나라도 SHAP > 0인 유전자 수: 475
['APOE', 'AASDH', 'CYYR1', 'SPDEF', 'FAM171B', 'ACAA1', 'SLC25A48', 'ABCA3', 'PAX3', 'ATG4D', 'ARTN', 'FAM129A', 'PHF1', 'ABTB2', 'SERPINA10', 'MORF4L2', 'ARHGAP26', 'TRMT5', 'CDHR2', 'ZNF257', 'C5orf20', 'ARL4D', 'A2ML1', 'AURKC', 'ACSS2', 'MYH4', 'ALDH8A1', 'THPO', 'MTHFD2', 'ELOVL5', 'ADRA2B', 'MYLIP', 'ATP2A2', 'KLRC1', 'EPHB1', 'ACO2', 'MRPS9', 'ZNF705D', 'ACTL7A', 'CXCR6', 'C8G', 'ZNF420', 'A1CF', 'C16orf78', 'B3GALT5', 'LPAL2', 'SFMBT2', 'PABPC4', 'NHS', 'C20orf71', 'ADAMTS4', 'SNX12', 'MRPL23', 'NDST4', 'NLRC3', 'ABCC6P2', 'ANKRD34C', 'KCP', 'DEFB134', 'AKAP1', 'DZIP1', 'RGS11', 'RPS15A', 'AAK1', 'ANKS1B', 'C10orf76', 'RPA4', 'SERPING1', 'SETD4', 'AP1G2', 'ZNF362', 'AAAS', 'FITM2', 'C1orf213', 'PAGE2', 'C12orf32', 'ABHD12B', 'CNOT4', 'NDUFAF3', 'TAF5L', 'C18orf45', 'CRISP1', 'TMEM25', 'ARHGAP11B', 'AMDHD1', 'ABCC1', 'NUTF2', 'ACVR2B', 'MRVI1', 'GPR141', 'SPRR3', 'HBG1', 'SNORD116-1', 'EHF', 'GZMK', 'IQCE', 'IMP3', 'ABCB8', 'LHB', 'ACSBG1', 'D

In [57]:
shap_df_sorted = shap_df.sort_values(by='SHAP_pure_fat_C_A', ascending=False).reset_index()

In [58]:
shap_df_sorted

,Gene,SHAP_weight_kg_C_A,SHAP_bmi_C_A,SHAP_fat_C_A,SHAP_pure_fat_C_A
0,HNRNPA1P10,0.000000,0.000000,0.0,0.519105
1,BTN2A1,0.343720,0.156273,0.0,0.473053
2,PTPRD,0.000000,0.000000,0.0,0.445951
3,SNORD116-1,0.002763,0.000000,0.0,0.425596
4,CDCA5,0.000000,0.000000,0.0,0.234879
...,...,...,...,...,...
20249,GBP7,0.000000,0.000000,0.0,0.000000
20250,GBP6,0.000000,0.000000,0.0,0.000000
20251,GBP5,0.000000,0.000000,0.0,0.000000
20252,GBP4,0.000000,0.000000,0.0,0.000000


# **SHAP 유전자 필터링 + Optuna 튜닝 + 교차검증**

In [45]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 13.8 MB/s eta 0:00:00


실수로 X-> X_common, y-> y_common과정 지워버렸음;;

In [69]:
# X_common과 y_common을 좌우로 합치기 (subject_id 기준)
merged_df = pd.concat([X_common, y_common], axis=1)

# 결측치가 있는 행 제거
merged_clean = merged_df.dropna()

# 다시 X, y로 분리
X_clean = merged_clean[X_common.columns]
y_clean = merged_clean[y_common.columns]

In [72]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
import optuna
import pandas as pd

In [ ]:
print(len(shap_weight_genes))
print(len(shap_bmi_genes))
print(len(shap_fat_pct_genes))
print(len(shap_fat_mass_genes))

In [76]:
shap_df.index

Index(['A1BG', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1',
       'AAAS', 'AACS',
       ...
       'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B', 'ZYX', 'ZZEF1', 'ZZZ3',
       'psiTPTE22', 'tAKR'],
      dtype='object', name='Gene', length=20254)

In [73]:
shap_df.columns

Index(['SHAP_weight_kg_C_A', 'SHAP_bmi_C_A', 'SHAP_fat_C_A',
       'SHAP_pure_fat_C_A'],
      dtype='object')

In [80]:
# ▶ 실험 A: SHAP_fat_mass 기준 상위 유전자 138개
selected_genes_1 = shap_df[shap_df['SHAP_pure_fat_C_A'] > 0]\
    .sort_values(by='SHAP_pure_fat_C_A', ascending=False).index.tolist()

X_filtered_1 = pd.DataFrame(X_clean, columns=genes)[selected_genes_1]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_filtered_1, y_clean, test_size=0.2, random_state=42)

# ▶ 실험 B: 모든 SHAP > 0 유전자 합집합 475개
selected_genes_2 = list(
    set(shap_weight_genes) | set(shap_bmi_genes) |
    set(shap_fat_pct_genes) | set(shap_fat_mass_genes)
)
X_filtered_2 = pd.DataFrame(X_clean, columns=genes)[selected_genes_2]
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_filtered_2, y_clean, test_size=0.2, random_state=42)

In [82]:
print(len(selected_genes_1))
print(len(selected_genes_2))

138
475


In [83]:
# Optuna 튜닝 함수
def optimize_model(X_train, y_train):
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 400),
            "max_depth": trial.suggest_int("max_depth", 3, 8),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0)
        }
        base_model = XGBRegressor(**params, random_state=42)
        model = MultiOutputRegressor(base_model)
        score = cross_val_score(model, X_train, y_train, cv=3, scoring="r2").mean()
        return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)
    return study.best_params

In [84]:
# A 모델 학습 및 평가
params_1 = optimize_model(X_train1, y_train1)
model_1 = MultiOutputRegressor(XGBRegressor(**params_1, random_state=42))
model_1.fit(X_train1, y_train1)
y_pred1 = model_1.predict(X_test1)
r2_1 = r2_score(y_test1, y_pred1)

[I 2025-06-16 15:50:24,792] A new study created in memory with name: no-name-1e59e74d-d7bd-4a68-a473-5a815757f34b
[I 2025-06-16 15:50:29,732] Trial 0 finished with value: 0.13524338603019714 and parameters: {'n_estimators': 156, 'max_depth': 5, 'learning_rate': 0.05023011475619976, 'subsample': 0.7272868137886211, 'colsample_bytree': 0.7216102224110019}. Best is trial 0 with value: 0.13524338603019714.
[I 2025-06-16 15:50:37,070] Trial 1 finished with value: 0.11676878730456035 and parameters: {'n_estimators': 211, 'max_depth': 8, 'learning_rate': 0.13895673490974675, 'subsample': 0.8587109803880826, 'colsample_bytree': 0.6151424421554843}. Best is trial 0 with value: 0.13524338603019714.
[I 2025-06-16 15:50:42,064] Trial 2 finished with value: 0.08969279130299886 and parameters: {'n_estimators': 358, 'max_depth': 4, 'learning_rate': 0.14779751526166798, 'subsample': 0.6418869987279306, 'colsample_bytree': 0.6677649669191131}. Best is trial 0 with value: 0.13524338603019714.
[I 2025-06

In [85]:
# B 모델 학습 및 평가
params_2 = optimize_model(X_train2, y_train2)
model_2 = MultiOutputRegressor(XGBRegressor(**params_2, random_state=42))
model_2.fit(X_train2, y_train2)
y_pred2 = model_2.predict(X_test2)
r2_2 = r2_score(y_test2, y_pred2)

[I 2025-06-16 15:55:42,708] A new study created in memory with name: no-name-c38d5a6f-565b-406a-95c8-1c9554da74aa
[I 2025-06-16 15:55:54,424] Trial 0 finished with value: -0.018105342984199524 and parameters: {'n_estimators': 173, 'max_depth': 8, 'learning_rate': 0.1901077059378496, 'subsample': 0.7385670390439851, 'colsample_bytree': 0.9617235301906714}. Best is trial 0 with value: -0.018105342984199524.
[I 2025-06-16 15:56:08,606] Trial 1 finished with value: 0.09461369117101033 and parameters: {'n_estimators': 211, 'max_depth': 8, 'learning_rate': 0.08145964106114051, 'subsample': 0.8429834595672858, 'colsample_bytree': 0.8693864027141758}. Best is trial 1 with value: 0.09461369117101033.
[I 2025-06-16 15:56:19,660] Trial 2 finished with value: 0.018663763999938965 and parameters: {'n_estimators': 148, 'max_depth': 7, 'learning_rate': 0.013123999547271326, 'subsample': 0.664652366273166, 'colsample_bytree': 0.9497000506006047}. Best is trial 1 with value: 0.09461369117101033.
[I 202

In [87]:
print("Model1 Best Parameters:", params_1)
print("Model2 Best Parameters:", params_2)

Model1 Best Parameters: {'n_estimators': 381, 'max_depth': 8, 'learning_rate': 0.054459218896571696, 'subsample': 0.7615324063508745, 'colsample_bytree': 0.9579766126464718}
Model2 Best Parameters: {'n_estimators': 142, 'max_depth': 7, 'learning_rate': 0.1497198437398045, 'subsample': 0.825869701275801, 'colsample_bytree': 0.6003946745227667}


In [86]:
print("A안 (fat_mass 기준 상위 유전자, 개수:", len(selected_genes_1), ") R²:", r2_1)
print("B안 (4개 SHAP 합집합 유전자, 개수:", len(selected_genes_2), ") R²:", r2_2)

A안 (fat_mass 기준 상위 유전자, 개수: 138 ) R²: 0.39047497510910034
B안 (4개 SHAP 합집합 유전자, 개수: 475 ) R²: 0.33257317543029785


A안 Model1: pure_fat_C_A기준 양수인 값을 가진 상위 유전자 138개를 feature로 했을 때 r2가 더 높게 나왔으므로 Model1을 이용하여 SHAP값 계산


In [94]:
import shap
import numpy as np
import pandas as pd

def compute_shap_df(model, X_input, target_names):
    shap_dict = {}

    for i, estimator in enumerate(model.estimators_):
        target_name = target_names[i]  # 예: 'weight_kg_C_A' 등
        explainer = shap.TreeExplainer(estimator)
        shap_values = explainer.shap_values(X_input)
        mean_abs_shap = np.abs(shap_values).mean(axis=0)
        shap_dict[target_name] = mean_abs_shap

    # SHAP값을 DataFrame으로
    shap_df = pd.DataFrame(shap_dict, index=X_input.columns).T
    return shap_df

In [95]:
target_names = y_train1.columns.tolist()

shap1_df = compute_shap_df(model_1, X_train1, target_names)
shap2_df = compute_shap_df(model_2, X_train2, target_names)

In [96]:
shap1_df

,HNRNPA1P10,BTN2A1,PTPRD,SNORD116-1,CDCA5,C8orf80,ARF5,ANP32B,SULT1C4,CBY1,...,ABAT,TRO,SFMBT2,SRCIN1,ADAMTS4,ABHD12B,ZNF705D,AFAP1L2,DLX3,ATP1A3
weight_kg_C_A,0.247319,0.103196,0.077105,0.201080,0.092313,0.009061,0.053822,0.012425,0.027417,0.038274,...,0.004355,0.002476,0.040468,0.014481,0.000937,0.007409,0.222945,0.000758,0.006048,0.002923
bmi_C_A,0.062802,0.018025,0.022044,0.044329,0.017000,0.004891,0.011383,0.001588,0.008996,0.009588,...,0.000105,0.000466,0.027148,0.001102,0.001640,0.000412,0.035223,0.000250,0.000079,0.006358
fat_C_A,0.315680,0.183632,0.079298,0.160215,0.053380,0.013559,0.059069,0.287340,0.037754,0.103328,...,0.001175,0.000232,0.013709,0.002584,0.003000,0.001980,0.002301,0.002685,0.029798,0.002862
pure_fat_C_A,0.431585,0.172992,0.069219,0.334195,0.146440,0.048397,0.086344,0.167967,0.091685,0.078083,...,0.011762,0.000474,0.048596,0.000120,0.001375,0.000016,0.117619,0.002408,0.002202,0.007281


In [97]:
shap2_df

,APOE,AASDH,CYYR1,SPDEF,FAM171B,ACAA1,SLC25A48,ABCA3,PAX3,ATG4D,...,GPR75,AFAP1L2,NPAS1,AGPAT6,C8orf42,MIR130A,ACBD7,AADACL4,PEX7,BTN2A1
weight_kg_C_A,0.0,0.0,0.0,0.040531,0.0,0.0,0.085087,0.0,0.031601,0.0,...,0.002156,0.004058,0.0,0.0,0.0,0.0,0.0,0.000068,0.0,0.144854
bmi_C_A,0.0,0.0,0.0,0.010240,0.0,0.0,0.010562,0.0,0.013593,0.0,...,0.002215,0.000551,0.0,0.0,0.0,0.0,0.0,0.000950,0.0,0.036673
fat_C_A,0.0,0.0,0.0,0.013938,0.0,0.0,0.023461,0.0,0.053204,0.0,...,0.000915,0.013769,0.0,0.0,0.0,0.0,0.0,0.000048,0.0,0.042822
pure_fat_C_A,0.0,0.0,0.0,0.028516,0.0,0.0,0.041111,0.0,0.008174,0.0,...,0.000149,0.000140,0.0,0.0,0.0,0.0,0.0,0.001035,0.0,0.000034


In [98]:
shap1_df.to_csv('/content/drive/MyDrive/BIO_MEDICAL_PROJECT/obesity_gene_analysis/data/shap1_df.csv')
shap2_df.to_csv('/content/drive/MyDrive/BIO_MEDICAL_PROJECT/obesity_gene_analysis/data/shap2_df.csv')